#### Environment

In [80]:
import pandas as pd

teams = pd.read_csv('mastering_ncaat_teams_historical.csv')
teams = teams[['team_id', 'kenpom_id']]
print('Teams: ', teams.shape)
teams.head()

Teams:  (1407, 2)


,team_id,kenpom_id
0,2004 Connecticut,2004 Connecticut
1,2004 Oklahoma St.,2004 Oklahoma St.
2,2004 Duke,2004 Duke
3,2004 Georgia Tech,2004 Georgia Tech
4,2004 Saint Joseph's,2004 Saint Joseph's


#### Kenpom

In [81]:
ratings = pd.read_csv('raw_kenpom_ratings_historical.csv')
ratings['kenpom_id'] = ratings['Season'].astype(str) + ' ' + ratings['TeamName']
ratings = ratings[['kenpom_id', 'AdjEM', 'AdjOE', 'AdjDE', 'OE', 'DE', 'Tempo', 'AdjTempo']]
print('Ratings: ', ratings.shape)

offense = pd.read_csv('raw_kenpom_offense_historical.csv')
offense['kenpom_id'] = offense['Season'].astype(str) + ' ' + offense['TeamName']
offense = offense[['kenpom_id', 'eFG_Pct', 'TO_Pct', 'OR_Pct', 'FT_Rate']]
print('Offense: ', offense.shape)

defense = pd.read_csv('raw_kenpom_defense_historical.csv')
defense['kenpom_id'] = defense['Season'].astype(str) + ' ' + defense['TeamName']
defense = defense[['kenpom_id', 'eFG_Pct', 'TO_Pct', 'OR_Pct', 'FT_Rate']]
print('Defense: ', defense.shape)

misc = pd.read_csv('raw_kenpom_misc_historical.csv')
misc['kenpom_id'] = misc['Season'].astype(str) + ' ' + misc['TeamName']
misc = misc[['kenpom_id', 'FG2Pct', 'FG3Pct', 'FTPct', 'BlockPct', 
             'OppFG2Pct', 'OppFG3Pct', 'OppFTPct', 'OppBlockPct', 
             'ARate', 'OppARate', 'StlRate', 'OppStlRate']]
print('Misc: ', misc.shape)

# Roster starts at 2007
#roster = pd.read_csv('raw_kenpom_roster_historical.csv')
#roster['kenpom_id'] = roster['Season'].astype(str) + ' ' + roster['TeamName']
#roster = roster[['kenpom_id', 'HgtEff', 'Size', 'Exp', 'Bench']]
#print('Roster: ', roster.shape)

Ratings:  (7950, 8)
Offense:  (7950, 5)
Defense:  (7950, 5)
Misc:  (7950, 13)


In [82]:
# Join the Kenpom tables for the 20 years models
join_1 = pd.merge(teams, ratings, on='kenpom_id', how='inner')
print('Join Ratings: ', join_1.shape)
join_2 = pd.merge(join_1, offense, on='kenpom_id', how='inner')
print('Join Offense: ', join_2.shape)
join_3 = pd.merge(join_2, defense, on='kenpom_id', how='inner')
print('Join Defense: ', join_3.shape)
join_4 = pd.merge(join_3, misc, on='kenpom_id', how='inner')
print('Join Misc: ', join_4.shape)

Join Ratings:  (1407, 9)
Join Offense:  (1407, 13)
Join Defense:  (1407, 17)
Join Misc:  (1407, 29)


In [83]:
kenpom_df = join_4.rename(columns={
                             'AdjEM': 'kenpom_adjem',
                             'AdjOE': 'kenpom_adjoe',
                             'AdjDE': 'kenpom_adjde',
                             'OE': 'off_eff',
                             'DE': 'def_eff',
                             'Tempo': 'tempo',
                             'AdjTempo': 'adj_tempo',
                             'eFG_Pct_x': 'off_efg_pct',
                             'TO_Pct_x': 'off_to_pct',
                             'OR_Pct_x': 'off_or_pct',
                             'FT_Rate_x': 'off_ft_rate',
                             'eFG_Pct_y': 'def_efg_pct',
                             'TO_Pct_y': 'def_to_pct',
                             'OR_Pct_y': 'def_or_pct',
                             'FT_Rate_y': 'def_ft_rate',
                             'FG2Pct': 'off_2pt_pct',
                             'FG3Pct': 'off_3pt_pct',
                             'FTPct': 'off_ft_pct',
                             'BlockPct': 'off_block_pct',
                             'OppFG2Pct': 'def_2pt_pct',
                             'OppFG3Pct': 'def_3pt_pct',
                             'OppFTPct': 'def_ft_pct',
                             'OppBlockPct': 'def_block_pct',
                             'ARate': 'off_ast_rate',
                             'OppARate': 'def_ast_rate',
                             'StlRate': 'off_stl_rate',
                             'OppStlRate': 'def_stl_rate'
})

#### Matchups

In [84]:
games = pd.read_csv('mastering_ncaat_games_historical.csv')
games = games[['year', 'region', 'round', 'high_bracket_seed', 'high_bracket_team', 'low_bracket_seed', 'low_bracket_team', 'win']]
print(games.shape)
games.head()

(1258, 8)


,year,region,round,high_bracket_seed,high_bracket_team,low_bracket_seed,low_bracket_team,win
0,2004,East,First Round,1,2004 Saint Joseph's,16,2004 Liberty,1
1,2004,East,First Round,8,2004 Texas Tech,9,2004 Charlotte,1
2,2004,East,First Round,5,2004 Florida,12,2004 Manhattan,0
3,2004,East,First Round,4,2004 Wake Forest,13,2004 VCU,1
4,2004,East,First Round,6,2004 Wisconsin,11,2004 Richmond,1


In [85]:
matchups_1 = pd.merge(games, kenpom_df, left_on='high_bracket_team', right_on='team_id', how='inner')
matchups_1 = matchups_1.drop(columns=['team_id', 'kenpom_id'])
print('High team matchup: ', matchups_1.shape)

High team matchup:  (1258, 35)


In [86]:
matchups_1 = matchups_1.rename(columns={
                             'kenpom_adjem': 'high_kenpom_adjem',
                             'kenpom_adjoe': 'high_kenpom_adjoe',
                             'kenpom_adjde': 'high_kenpom_adjde',
                             'off_eff': 'high_off_eff',
                             'def_eff': 'high_def_eff',
                             'tempo': 'high_tempo',
                             'adj_tempo': 'high_adj_tempo',
                             'off_efg_pct': 'high_off_efg_pct',
                             'off_to_pct': 'high_off_to_pct',
                             'off_or_pct': 'high_off_or_pct',
                             'off_ft_rate': 'high_off_ft_rate',
                             'def_efg_pct': 'high_def_efg_pct',
                             'def_to_pct': 'high_def_to_pct',
                             'def_or_pct': 'high_def_or_pct',
                             'def_ft_rate': 'high_def_ft_rate',
                             'off_2pt_pct': 'high_off_2pt_pct',
                             'off_3pt_pct': 'high_off_3pt_pct',
                             'off_ft_pct': 'high_off_ft_pct',
                             'off_block_pct': 'high_off_block_pct',
                             'def_2pt_pct': 'high_def_2pt_pct',
                             'def_3pt_pct': 'high_def_3pt_pct',
                             'def_ft_pct': 'high_def_ft_pct',
                             'def_block_pct': 'high_def_block_pct',
                             'off_ast_rate': 'high_off_ast_rate',
                             'def_ast_rate': 'high_def_ast_rate',
                             'off_stl_rate': 'high_off_stl_rate',
                             'def_stl_rate': 'high_def_stl_rate'
})

In [87]:
matchups_1['high_kenpom_adjde'] = 130 - matchups_1['high_kenpom_adjde']
matchups_1['high_def_eff'] = 130 - matchups_1['high_def_eff']
matchups_1['high_off_to_pct'] = 100 - matchups_1['high_off_to_pct']
matchups_1['high_def_efg_pct'] = 100 - matchups_1['high_def_efg_pct']
matchups_1['high_def_or_pct'] = 100 - matchups_1['high_def_or_pct']
matchups_1['high_def_ft_rate'] = 100 - matchups_1['high_def_ft_rate']
matchups_1['high_off_block_pct'] = 100 - matchups_1['high_off_block_pct']
matchups_1['high_def_2pt_pct'] = 100 - matchups_1['high_def_2pt_pct']
matchups_1['high_def_3pt_pct'] = 100 - matchups_1['high_def_3pt_pct']
matchups_1['high_def_ft_pct'] = 100 - matchups_1['high_def_ft_pct']
matchups_1['high_def_ast_rate'] = 100 - matchups_1['high_def_ast_rate']
matchups_1['high_off_stl_rate'] = 100 - matchups_1['high_off_stl_rate']

In [88]:
matchups_2 = pd.merge(matchups_1, kenpom_df, left_on='low_bracket_team', right_on='team_id', how='inner')
matchups_2 = matchups_2.drop(columns=['team_id', 'kenpom_id'])
print('Low team matchup: ', matchups_2.shape)

Low team matchup:  (1258, 62)


In [89]:
matchups_2 = matchups_2.rename(columns={
                             'kenpom_adjem': 'low_kenpom_adjem',
                             'kenpom_adjoe': 'low_kenpom_adjoe',
                             'kenpom_adjde': 'low_kenpom_adjde',
                             'off_eff': 'low_off_eff',
                             'def_eff': 'low_def_eff',
                             'tempo': 'low_tempo',
                             'adj_tempo': 'low_adj_tempo',
                             'off_efg_pct': 'low_off_efg_pct',
                             'off_to_pct': 'low_off_to_pct',
                             'off_or_pct': 'low_off_or_pct',
                             'off_ft_rate': 'low_off_ft_rate',
                             'def_efg_pct': 'low_def_efg_pct',
                             'def_to_pct': 'low_def_to_pct',
                             'def_or_pct': 'low_def_or_pct',
                             'def_ft_rate': 'low_def_ft_rate',
                             'off_2pt_pct': 'low_off_2pt_pct',
                             'off_3pt_pct': 'low_off_3pt_pct',
                             'off_ft_pct': 'low_off_ft_pct',
                             'off_block_pct': 'low_off_block_pct',
                             'def_2pt_pct': 'low_def_2pt_pct',
                             'def_3pt_pct': 'low_def_3pt_pct',
                             'def_ft_pct': 'low_def_ft_pct',
                             'def_block_pct': 'low_def_block_pct',
                             'off_ast_rate': 'low_off_ast_rate',
                             'def_ast_rate': 'low_def_ast_rate',
                             'off_stl_rate': 'low_off_stl_rate',
                             'def_stl_rate': 'low_def_stl_rate'
})

In [90]:
matchups_2['low_kenpom_adjde'] = 130 - matchups_2['low_kenpom_adjde']
matchups_2['low_def_eff'] = 130 - matchups_2['low_def_eff']
matchups_2['low_off_to_pct'] = 100 - matchups_2['low_off_to_pct']
matchups_2['low_def_efg_pct'] = 100 - matchups_2['low_def_efg_pct']
matchups_2['low_def_or_pct'] = 100 - matchups_2['low_def_or_pct']
matchups_2['low_def_ft_rate'] = 100 - matchups_2['low_def_ft_rate']
matchups_2['low_off_block_pct'] = 100 - matchups_2['low_off_block_pct']
matchups_2['low_def_2pt_pct'] = 100 - matchups_2['low_def_2pt_pct']
matchups_2['low_def_3pt_pct'] = 100 - matchups_2['low_def_3pt_pct']
matchups_2['low_def_ft_pct'] = 100 - matchups_2['low_def_ft_pct']
matchups_2['low_def_ast_rate'] = 100 - matchups_2['low_def_ast_rate']
matchups_2['low_off_stl_rate'] = 100 - matchups_2['low_off_stl_rate']

In [94]:
matchups_df = matchups_2[['year',
                          'region',
                          'round',
                          'high_bracket_seed',
                          'high_bracket_team',
                          'low_bracket_seed',
                          'low_bracket_team',
                          'win']]
# Efficiency
matchups_df['kenpom_adjem'] = matchups_2['high_kenpom_adjem'] - matchups_2['low_kenpom_adjem']
matchups_df['kenpom_adjoe'] = matchups_2['high_kenpom_adjoe'] - matchups_2['low_kenpom_adjde']
matchups_df['kenpom_adjde'] = matchups_2['high_kenpom_adjde'] - matchups_2['low_kenpom_adjoe']
matchups_df['off_eff'] = matchups_2['high_off_eff'] - matchups_2['high_def_eff']
matchups_df['def_eff'] = matchups_2['high_def_eff'] - matchups_2['high_off_eff']
matchups_df['tempo'] = matchups_2['high_tempo'] - matchups_2['low_tempo']
matchups_df['adj_tempo'] = matchups_2['high_adj_tempo'] - matchups_2['low_adj_tempo']
# Four Factors Offense
matchups_df['off_efg_pct'] = matchups_2['high_off_efg_pct'] - matchups_2['low_def_efg_pct']
matchups_df['off_to_pct'] = matchups_2['high_off_to_pct'] - matchups_2['low_def_to_pct']
matchups_df['off_or_pct'] = matchups_2['high_off_or_pct'] - matchups_2['low_def_or_pct']
matchups_df['off_ft_rate'] = matchups_2['high_off_ft_rate'] - matchups_2['low_def_ft_rate']
# Four Factors Defense
matchups_df['def_efg_pct'] = matchups_2['high_def_efg_pct'] - matchups_2['low_off_efg_pct']
matchups_df['def_to_pct'] = matchups_2['high_def_to_pct'] - matchups_2['low_off_to_pct']
matchups_df['def_or_pct'] = matchups_2['high_def_or_pct'] - matchups_2['low_off_or_pct']
matchups_df['def_ft_rate'] = matchups_2['high_def_ft_rate'] - matchups_2['low_off_ft_rate']
# Traditional Stats Offense
matchups_df['off_2pt_pct'] = matchups_2['high_off_2pt_pct'] - matchups_2['low_def_2pt_pct']
matchups_df['off_3pt_pct'] = matchups_2['high_off_3pt_pct'] - matchups_2['low_def_3pt_pct']
matchups_df['off_ft_pct'] = matchups_2['high_off_ft_pct'] - matchups_2['low_def_ft_pct']
matchups_df['off_block_pct'] = matchups_2['high_off_block_pct'] - matchups_2['low_def_block_pct']
# Traditional Stats Defense
matchups_df['def_2pt_pct'] = matchups_2['high_def_2pt_pct'] - matchups_2['low_off_2pt_pct']
matchups_df['def_3pt_pct'] = matchups_2['high_def_3pt_pct'] - matchups_2['low_off_3pt_pct']
matchups_df['def_ft_pct'] = matchups_2['high_def_ft_pct'] - matchups_2['low_off_ft_pct']
matchups_df['def_block_pct'] = matchups_2['high_def_block_pct'] - matchups_2['low_off_block_pct']
# Rates
matchups_df['off_ast_rate'] = matchups_2['high_off_ast_rate'] - matchups_2['low_def_ast_rate']
matchups_df['off_stl_rate'] = matchups_2['high_off_stl_rate'] - matchups_2['low_def_stl_rate']
matchups_df['def_ast_rate'] = matchups_2['high_def_ast_rate'] - matchups_2['low_off_ast_rate']
matchups_df['def_stl_rate'] = matchups_2['high_def_stl_rate'] - matchups_2['low_off_stl_rate']

/var/folders/nd/7h3lcr8d2cjbxfmfqghczqz40000gn/T/ipykernel_23075/2550605524.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchups_df['kenpom_adjem'] = matchups_2['high_kenpom_adjem'] - matchups_2['low_kenpom_adjem']
/var/folders/nd/7h3lcr8d2cjbxfmfqghczqz40000gn/T/ipykernel_23075/2550605524.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchups_df['kenpom_adjoe'] = matchups_2['high_kenpom_adjoe'] - matchups_2['low_kenpom_adjde']
/var/folders/nd/7h3lcr8d2cjbxfmfqghczqz40000gn/T/ipykernel_23075

In [95]:
print(matchups_df.shape)
matchups_df.head()

(1258, 35)


,year,region,round,high_bracket_seed,high_bracket_team,low_bracket_seed,low_bracket_team,win,kenpom_adjem,kenpom_adjoe,...,off_ft_pct,off_block_pct,def_2pt_pct,def_3pt_pct,def_ft_pct,def_block_pct,off_ast_rate,off_stl_rate,def_ast_rate,def_stl_rate
0,2004,East,First Round,1,2004 Saint Joseph's,16,2004 Liberty,1,30.51285,89.1570,...,33.5825,81.9021,5.4501,41.7977,-37.4861,-84.6118,13.7502,99.7470,-10.0856,-99.7788
1,2004,East,Second Round,1,2004 Saint Joseph's,8,2004 Texas Tech,1,11.71310,82.5819,...,36.8050,84.5246,5.1594,38.3150,-43.5228,-83.9574,10.0373,99.7747,-8.3934,-99.7847
2,2004,East,Sweet 16,1,2004 Saint Joseph's,4,2004 Wake Forest,1,5.43710,86.9808,...,40.3362,81.3147,2.6608,33.3825,-40.5819,-84.6767,14.3976,99.7599,-8.8366,-99.7824
3,2004,East,Second Round,12,2004 Manhattan,4,2004 Wake Forest,0,-10.75180,76.1598,...,46.2060,79.1674,2.6516,29.1452,-40.8626,-81.4092,-1.5225,99.7573,-5.7294,-99.7894
4,2004,East,Elite Eight,1,2004 Saint Joseph's,2,2004 Oklahoma St.,0,3.01610,81.0911,...,35.9231,84.1024,-0.9003,35.1029,-38.7064,-83.5712,15.0771,99.7750,-8.9764,-99.7899


In [96]:
matchups_df.to_csv('matchups_20.csv')